# Heat Equation in 1D (Inverse Problem)

We will solve a heat equation in 1D:

$$
\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}, \qquad \text{where} \quad x  \in [0,1],  \quad t \in [0,1],
$$

where $\alpha$ is estimated using the exact solution.

The Dirichlet boundary conditions:

$$
u(0, t) = u(1,t) = 0
$$

and periodic initial condition:

$$
u(x,0) = sin(\frac{\pi x}{L}), 
$$

where $L = 1$  is the length of the bar.

The exact solution is $u(x,t) = e^{\frac{-\pi^2 \alpha t}{L^2}}sin(\frac{\pi x}{L})$.

## Implementation and Training

First, we import the libraries:

In [36]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import datetime
import math

We define the $sine$ and $euler$ functions and the values of $\pi, \alpha$ and $L$

In [37]:
sin = torch.sin
e = torch.exp
pi = math.pi
L = 1
alpha = 0.4

and define the exact solution function.

In [38]:
def sol(x, t):
    return e(-(pi**2)*alpha*t/(L**2))*sin(pi*x/L)

Now, we set the parameters of the neural network: it has a structure with 2 inputs ($x, t$) and 1 output ($u(x,t)$), 15000 maximum training steps, 3 hidden layers with 32 neurons each, 100 samples and a target minimum loss value of $10^{-4}$.

In [39]:
inputs = 2
outputs = 1
hn_1 = 32
hn_2 = 32
hn_3 = 32
steps = 0
max_steps = 10000
loss = 10
min_loss = 1e-5
log_each = 500
samples = 100
loss_values = []

In this part, we define a new class implementing the activation function $sin(x)$, due to the oscillatory nature of the solution.

In [40]:
class Sine(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.sin(x)

To solve the inverse problem, we introduce a parameter that estimates the value of $\alpha$.

In [41]:
ALPHA = nn.Parameter(torch.tensor(1.0, requires_grad=True))

The multilayer perceptron (MLP) structure is:

In [42]:
mlp = nn.Sequential(
    nn.Linear(inputs,hn_1),
    Sine(),
    nn.Linear(hn_1, hn_2),
    Sine(),
    nn.Linear(hn_3, hn_3),
    Sine(),
    nn.Linear(hn_3, outputs)
)

optimizer = torch.optim.Adam(list(mlp.parameters()) + [ALPHA])
criterion = nn.MSELoss()
mlp.train()

Sequential(
  (0): Linear(in_features=2, out_features=32, bias=True)
  (1): Sine()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): Sine()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): Sine()
  (6): Linear(in_features=32, out_features=1, bias=True)
)

In this section, we define a function that calculates the gradients.

In [43]:
def computeGrads(y, x):
    grads, = torch.autograd.grad(y, x, grad_outputs=y.data.new(y.shape).fill_(1), create_graph=True, only_inputs=True)
    return grads

Next, we define the main training loop and the timer. We use the exact solution in the test domain to calculate the data loss.

In [44]:
starttime_train = datetime.datetime.now()
print('----Training Started----')

while steps < max_steps and loss > min_loss:
    x = torch.rand(samples)
    t = torch.rand(samples)
    X = torch.stack([x, t], axis=-1)
    X.requires_grad = True
    Y = mlp(X)
    grads = computeGrads(Y, X)
    dudx = grads[:, :1]
    dudt = grads[:, 1:]
    grads2 = computeGrads(dudx, X)
    d2udx2 = grads2[:, :1]
    pde_loss = criterion(dudt, ALPHA*d2udx2)

    #initial condition
    t0 = torch.zeros(samples)
    X_initial = torch.stack([x, t0], axis=-1)
    Y_initial = mlp(X_initial)
    u_initial = sin(pi*x/L).unsqueeze(1)
    ic_loss = criterion(Y_initial, u_initial)

    #boundary conditions x=0
    x0 = torch.zeros(samples)
    X_boundary0 = torch.stack([x0, t], axis=-1)
    Y_boundary0 = mlp(X_boundary0)
    zero = torch.zeros(samples, 1)
    bc0_loss = criterion(Y_boundary0, zero)
    
    #boundary conditions x=1
    x1 = torch.ones(samples)
    X_boundary1 = torch.stack([x1, t], axis=-1)
    Y_boundary1 = mlp(X_boundary1)
    bc1_loss = criterion(Y_boundary1, zero)

    bc_loss = bc0_loss + bc1_loss

    #data
    data_loss = criterion(Y, sol(x,t).unsqueeze(1))

    optimizer.zero_grad()
    loss = pde_loss + ic_loss  + bc_loss + data_loss
    loss.backward()
    optimizer.step()

    loss_values.append(loss.detach().numpy())

    if steps % log_each == 0:
        print(f'Step:{steps}| pde_loss {pde_loss.item():.5f} ic_loss {ic_loss.item():.5f} bc_loss {bc_loss.item():.5f} data_loss {data_loss.item():.5f}')

    steps+=1

endtime_train = datetime.datetime.now()
train_time = endtime_train - starttime_train
train_time_formatted = train_time.seconds + train_time.microseconds / 1e6
print('---Training Finished---')

print(f'Training Duration: {steps} steps in {train_time_formatted:.3f} seconds')

----Training Started----
Step:0| pde_loss 0.00290 ic_loss 0.63493 bc_loss 0.00041 data_loss 0.10335
Step:500| pde_loss 0.01317 ic_loss 0.04328 bc_loss 0.03142 data_loss 0.00441
Step:1000| pde_loss 0.00235 ic_loss 0.00160 bc_loss 0.00232 data_loss 0.00097
Step:1500| pde_loss 0.00106 ic_loss 0.00060 bc_loss 0.00095 data_loss 0.00069
Step:2000| pde_loss 0.00079 ic_loss 0.00037 bc_loss 0.00050 data_loss 0.00036
Step:2500| pde_loss 0.00055 ic_loss 0.00016 bc_loss 0.00030 data_loss 0.00011
Step:3000| pde_loss 0.00047 ic_loss 0.00005 bc_loss 0.00014 data_loss 0.00007
Step:3500| pde_loss 0.00022 ic_loss 0.00003 bc_loss 0.00005 data_loss 0.00007
Step:4000| pde_loss 0.00025 ic_loss 0.00002 bc_loss 0.00003 data_loss 0.00004
Step:4500| pde_loss 0.00007 ic_loss 0.00002 bc_loss 0.00001 data_loss 0.00002
Step:5000| pde_loss 0.00017 ic_loss 0.00002 bc_loss 0.00002 data_loss 0.00001
Step:5500| pde_loss 0.00008 ic_loss 0.00000 bc_loss 0.00001 data_loss 0.00000
Step:6000| pde_loss 0.00013 ic_loss 0.00001

## Visualization

First, we display the value of $\alpha$ estimated by the PINN.

In [45]:
print(f'{ALPHA=}')

ALPHA=Parameter containing:
tensor(0.3987, requires_grad=True)


To visualize the model solution, we generate the output of the trained model and display it as a GIF alongside the exact solution.

In [ ]:
x = torch.linspace(0, 1, samples)
t = torch.linspace(0, 1, samples)

u = [] 
ref_sol = []
time = []

for t_ in t:
    with torch.no_grad():
        X = torch.stack([x, torch.ones(samples)*t_], axis=-1)
        Y = mlp(X)
    u.append(Y.detach().numpy())
    ref_sol.append(sol(x, t_))
    time.append(t_)

def update(i):
    ax.clear()
    ax.plot(x, u[i], '.-', label = fr'PINN ($ \alpha = {ALPHA.detach().numpy().item():.5f} $)')
    ax.plot(x, ref_sol[i], '-', label = fr'Exact ($ \alpha = {alpha} $)')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$u(x,t)$')
    ax.set_title(f'$t = {time[i]:.3f}$')
    ax.set_ylim(-0.1, 1.1)
    ax.grid(True)
    ax.legend()
    return ax

fig = plt.figure(dpi=100)
ax = plt.subplot(1,1,1)
anim = animation.FuncAnimation(fig, update, frames=len(u), interval=200)
# anim.save('heat_inverse.gif', dpi=300, writer=PillowWriter(fps=25))
plt.close(fig)

[Animation](https://github.com/munozmfrancisco/PINNs/raw/main/GIFs/Inverse/heat_inverse.gif)